# LangChain Notion Agent Walkthrough

This notebook demonstrates how to compose `NotionSearchTool` and `NotionWriteTool`
inside an async feedback loop. Configure `NOTION_API_TOKEN` before running.


In [ ]:
import asyncio

from langchain_notion_tools import NotionSearchTool, NotionWriteTool, from_text


async def main():
    search = NotionSearchTool()
    write = NotionWriteTool()
    hits = search.run(query="design review")
    if not hits:
        print('No matching pages found')
        return
    target = hits[0]
    blocks = from_text("### Decision\n- Adopt shared design system")
    preview = write.run(
        update={"page_id": target['id'], "mode": "append"},
        blocks=blocks,
        is_dry_run=True,
    )
    print(preview['summary'])
    confirm = input('Post to Notion? [y/N] ').strip().lower() == 'y'
    if not confirm:
        return
    result = await write.arun(
        update={"page_id": target['id'], "mode": "append"},
        blocks=blocks,
    )
    print(result['summary'])
    print('URL:', result['url'])

asyncio.run(main())
